In [1]:
import pathlib
import xml.etree.ElementTree as ET

import pandas as pd
import numpy as np


data_path = pathlib.Path("..\\data")

# Read data

In [2]:
root = ET.parse(data_path / "Half-Life_-_WON_Easy.lss").getroot()

In [3]:
runs = (
    pd.DataFrame(
        data=[element.attrib for element in root.find("AttemptHistory").findall("Attempt")]
    )
    .set_index("id")
    .astype({"isStartedSynced": bool, "isEndedSynced": bool})
)
runs.loc[:, "started"] = pd.to_datetime(runs.loc[:, "started"])
runs.loc[:, "ended"] = pd.to_datetime(runs.loc[:, "ended"])
runs.head()

,started,isStartedSynced,ended,isEndedSynced
id,,,,
1,2018-01-17 17:45:30,True,2018-01-17 17:45:38,True
2,2018-01-17 17:45:38,True,2018-01-17 17:46:11,True
3,2018-01-17 17:46:11,True,2018-01-17 17:47:00,True
4,2018-01-17 17:47:05,True,2018-01-17 17:48:11,True
5,2018-01-17 17:48:11,True,2018-01-17 17:48:21,True


In [4]:
segments = []
for segment in root.find("Segments").findall("Segment"):
    name = segment.find("Name").text
    for segment_time in segment.find("SegmentHistory").findall("Time"):
        try:
            real_time = pd.to_timedelta(segment_time.find("RealTime").text)
        except AttributeError:
            real_time = np.nan
        try:
            game_time = pd.to_timedelta(segment_time.find("GameTime").text)
        except AttributeError:
            game_time = np.nan
        segments.append(
            {
                "Name": name,
                "id": int(segment_time.attrib["id"]),
                "RealTime": real_time,
                "GameTime": game_time
            }
        )
segments = pd.DataFrame(data=segments)
segments.head()

,Name,id,RealTime,GameTime
0,Anomalous Materials,700,00:04:29.376000,00:04:22.567000
1,Anomalous Materials,699,00:04:14.128000,00:04:11.970000
2,Anomalous Materials,697,00:04:14.768000,00:04:12.689000
3,Anomalous Materials,696,00:04:16.819000,00:04:14.321000
4,Anomalous Materials,1375,00:04:21.312000,00:04:18.196000


In [5]:
segments.Name.unique()

array(['Anomalous Materials', 'Unforeseen Consequences', 'Office Complex',
       '"We\'ve Got Hostiles"', 'Blast Pit', 'Power Up', 'On A Rail',
       'Apprehension', 'Residue Processing', 'Questionable Ethics',
       'Surface Tension', '"Forget About Freeman!" ', 'Lambda Core',
       'Xen', "Gonarch's Lair", 'Interloper', 'Nihilanth'], dtype=object)